In [44]:
def read_file(filepath):
    with open (filepath) as f:
        str_text = f.read()
    return str_text

In [45]:
#read_file('moby_dick_four_chapters.txt')

In [46]:
import spacy

In [47]:
nlp = spacy.load('en_core_web_sm',disable=['parser','tagger','ner'])

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [48]:
nlp.max_length = 1198623

In [49]:
def seperate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n']

In [50]:
d = read_file('/kaggle/input/mobydick-text/moby_dick_four_chapters.txt')

In [51]:
tokens = seperate_punc(d)

/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [52]:
len(tokens)

11338

In [53]:
train_len = 25 + 1

text_sequences = []

for i in range(train_len,len(tokens)):
    seq = tokens[i - train_len:i]

    text_sequences.append(seq)

In [54]:
train_len

26

In [55]:
type(text_sequences)

list

In [56]:
' '.join(text_sequences[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [57]:
' '.join(text_sequences[1])

'me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore'

In [58]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [59]:
tokenizer = Tokenizer()

In [60]:
tokenizer.fit_on_texts(text_sequences)

In [61]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [62]:
#sequences[0]

In [63]:
#tokenizer.index_word

In [64]:
for i in sequences[0]:
    print(f"{i} : {tokenizer.index_word[i]}")

956 : call
14 : me
263 : ishmael
51 : some
261 : years
408 : ago
87 : never
219 : mind
129 : how
111 : long
954 : precisely
260 : having
50 : little
43 : or
38 : no
314 : money
7 : in
23 : my
546 : purse
3 : and
150 : nothing
259 : particular
6 : to
2713 : interest
14 : me
24 : on


In [65]:
vocabulary = len(tokenizer.word_counts)

In [66]:
#tokenizer.word_counts

In [67]:
vocabulary

2718

In [68]:
import numpy as np

In [69]:
sequences = np.array(sequences)

In [70]:
sequences

array([[ 956,   14,  263, ..., 2713,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2712, ...,   53,    2, 2718],
       [ 166, 2712,    3, ...,    2, 2718,   26]])

In [71]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [72]:
X = sequences[:,:-1]

In [73]:
y = sequences[:,-1]

In [74]:
y = to_categorical(y,num_classes=vocabulary+1)

In [75]:
y[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [76]:
seq_len = X.shape[1]

In [77]:
X.shape

(11312, 25)

In [78]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

In [79]:
vocabulary = 2718
seq_len = 25

model = Sequential()
model.add(Embedding(input_dim = vocabulary+1,output_dim = 25,input_length=seq_len))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))

model.add(Dense(vocabulary+1,activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

model.build(input_shape=(None, seq_len))

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 25, 25)              │          67,975 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 25, 50)              │          15,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 50)                  │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 50)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2719)                │         138,669 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 244,594 (955.45 KB)

 Trainable params: 244,594 (955.45 KB)

 Non-trainable params: 0 (0.00 B)

In [80]:
from pickle import dump, load

In [81]:
model.fit(X,y,batch_size=128,epochs=200,verbose=1)

Epoch 1/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.0380 - loss: 7.4106
Epoch 2/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0545 - loss: 6.3042
Epoch 3/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.0526 - loss: 6.2669
Epoch 4/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0544 - loss: 6.2316
Epoch 5/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0539 - loss: 6.1470
Epoch 6/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0565 - loss: 6.0187
Epoch 7/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0516 - loss: 6.0000
Epoch 8/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0557 - loss: 5.9173
Epoch 9/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0638 - loss: 5.8490
Epoch 10/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0617 - loss: 5.7847
Epoch 11/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0634 - loss: 5.7648
Epoch 12/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy:

In [82]:
model.save('moby_dick_model.keras')

In [83]:
dump(tokenizer,open('my_simpletokenizer','wb'))

In [84]:
from keras.preprocessing.sequence import pad_sequences

In [85]:
def generate_text(model,tokenizer,seed_text,seq_len,num_gen_words):
    output_text = []

    input_text = seed_text
    print(input_text)

    for i in range(num_gen_words):

        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        pad_encoded = pad_sequences([encoded_text],maxlen=seq_len,truncating='pre')

        predictions = model.predict(pad_encoded, verbose=0)

        pre_word_ind = np.argmax(predictions[0])
        
        pre_word = tokenizer.index_word[pre_word_ind]

        input_text +=  ' ' + pre_word

        output_text.append(pre_word)

    return ' '.join(output_text)
    

In [88]:
import random
random.seed(101)
random_pick = random.randint(0,len(text_sequences))

random_seed_text = text_sequences[random_pick]

random_seed_text

seed_text = ' '.join(random_seed_text)

seed_text

"thought i to myself the man 's a human being just as i am he has just as much reason to fear me as i have"

In [92]:
output_text = []

input_text = seed_text
print(input_text)

for i in range(20):

    encoded_text = tokenizer.texts_to_sequences([input_text])[0]
    
    pad_encoded = pad_sequences([encoded_text],maxlen=seq_len,truncating='pre')

    predictions = model.predict(pad_encoded, verbose=0)

    pre_word_ind = np.argmax(predictions[0])
    
    pre_word = tokenizer.index_word[pre_word_ind]

    input_text +=  ' ' + pre_word

    output_text.append(pre_word)

thought i to myself the man 's a human being just as i am he has just as much reason to fear me as i have


In [ ]:
predictions.shape

In [ ]:
predictions[0].shape

In [ ]:
text_sequences[0]

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model('/kaggle/input/lstm_generator/tensorflow2/default/1/epochBIG.h5')

In [ ]:
model.summary()